## Problem 3

### Part a) 
Provide P(1), P(2), S(1), and S(2)

In [ ]:
import numpy as np

In [26]:
# Create matrices W1 and W2
W1 = np.array([[1.00, 0.50, 0.30, 0.10, 0.10],
               [0.50, 1.00, 0.40, 0.10, 0.10],
               [0.30, 0.40, 1.00, 0.30, 0.30],
               [0.10, 0.10, 0.30, 1.00, 0.50],
               [0.10, 0.10, 0.30, 0.50, 1.00]])

W2 = np.array([[1.00, 0.20, 0.50, 0.10, 0.10],
               [0.20, 1.00, 0.30, 0.10, 0.10],
               [0.50, 0.30, 1.00, 0.30, 0.30],
               [0.10, 0.10, 0.30, 1.00, 0.50],
               [0.10, 0.10, 0.30, 0.50, 1.00]])

# Create the normalized weight matrix P1 and P2
def normalized_weight_matrix(weight_matrix):
    W = weight_matrix
    P = np.empty((W.shape[0], W.shape[1]))
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            if(j == i):
                P[i,j] = 0.5 
            else: 
                sum = 0
                for k in range(W.shape[0]):
                    if k != i:
                        sum += W[i,k]
                P[i,j] = (W[i,j]) / (2 * sum)
    return P

Thus we have the matrices P1 and P2:

In [27]:
print("Matrix P1:") 
print(normalized_weight_matrix(W1))
print("Matrix P2:")
print(normalized_weight_matrix(W2))

Matrix P1:
[[0.5        0.25       0.15       0.05       0.05      ]
 [0.22727273 0.5        0.18181818 0.04545455 0.04545455]
 [0.11538462 0.15384615 0.5        0.11538462 0.11538462]
 [0.05       0.05       0.15       0.5        0.25      ]
 [0.05       0.05       0.15       0.25       0.5       ]]
Matrix P2:
[[0.5        0.11111111 0.27777778 0.05555556 0.05555556]
 [0.14285714 0.5        0.21428571 0.07142857 0.07142857]
 [0.17857143 0.10714286 0.5        0.10714286 0.10714286]
 [0.05       0.05       0.15       0.5        0.25      ]
 [0.05       0.05       0.15       0.25       0.5       ]]


In [28]:
# Similarity Matrix

def similarity_matrix(weight_matrix):
    W = weight_matrix
    S = np.empty((W.shape[0], W.shape[1]))
    # Loop through the weight matrix
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            # determine the nearest neighbors for data point i and their sum
            nearest_neighbours = k_nearest_neighbour(W[i], 2)
            sum_of_neighbours = 0
            
            for k in range(len(nearest_neighbours)):
                sum_of_neighbours += nearest_neighbours[k]
                
            if W[i,j] in nearest_neighbours:
                S[i,j] = W[i,j] / sum_of_neighbours
                 
            else:
                S[i,j] = 0
    
    return S

def k_nearest_neighbour(data, k):
    nearest_neighbours = np.empty(k)
    sorted_data = sorted(data, reverse=True)
    return sorted_data[:k]

Similarity Matrix S1 and S2:

In [29]:
print("S1: ")
print(similarity_matrix(W1))
print("S2: ")
print(similarity_matrix(W2))

S1: 
[[0.66666667 0.33333333 0.         0.         0.        ]
 [0.33333333 0.66666667 0.         0.         0.        ]
 [0.         0.28571429 0.71428571 0.         0.        ]
 [0.         0.         0.         0.66666667 0.33333333]
 [0.         0.         0.         0.33333333 0.66666667]]
S2: 
[[0.66666667 0.         0.33333333 0.         0.        ]
 [0.         0.76923077 0.23076923 0.         0.        ]
 [0.33333333 0.         0.66666667 0.         0.        ]
 [0.         0.         0.         0.66666667 0.33333333]
 [0.         0.         0.         0.33333333 0.66666667]]


### Part b) 
Perform two steps of the similarity network fusion method (i.e., compute P(1) 1 , P(2) 1 , P(1) 2 , and P(2) 2 as well as the corresponding P(c)s).

In [30]:
# Create the joint similarity matrix

def joint_similarity_matrix_time_steps(list_of_weight_matrices, time_steps):
    num_row = list_of_weight_matrices[0].shape[0]
    num_col = list_of_weight_matrices[0].shape[1]
    num_matrices = len(list_of_weight_matrices)
    
    # Initialize P and S matrices
    P1 = normalized_weight_matrix(list_of_weight_matrices[0])
    P2 = normalized_weight_matrix(list_of_weight_matrices[1])
    S1 = similarity_matrix(list_of_weight_matrices[0])
    S2 = similarity_matrix(list_of_weight_matrices[1])

     # Initalize P and S at time  and P_c at time t
    P1_t = []
    P2_t = []
    PC_t = []

    # Fill lists for time = 0
    P1_t.append(P1)
    P2_t.append(P2)
    PC_t.append((P1_t[0] + P2_t[0])/2)


    # Calculate next time step
    for i in range(1, time_steps + 1):
        P1_t.append(np.dot(S1, np.dot(P2_t[i-1], S1.T)))
        P2_t.append(np.dot(S2, np.dot(P1_t[i-1], S2.T)))
        PC_t.append((P1_t[i] + P2_t[i])/2)

        print("P1 at time step " + str(i) + ": " + str(P1_t[i]))
        print("P2 at time step " + str(i) + ": " + str(P2_t[i]))
        print("PC at time step " + str(i) + ": " + str(PC_t[i]))

joint_similarity_matrix_time_steps([W1, W2], 2)

P1 at time step 1: [[0.33421517 0.28747795 0.25207861 0.06084656 0.06084656]
 [0.29805996 0.33421517 0.27399849 0.06613757 0.06613757]
 [0.18537415 0.20238095 0.36151603 0.09693878 0.09693878]
 [0.05       0.05       0.12142857 0.38888889 0.36111111]
 [0.05       0.05       0.12142857 0.36111111 0.38888889]]
P2 at time step 1: [[0.33675214 0.22919132 0.3017094  0.07179487 0.07179487]
 [0.21938318 0.38207059 0.23731397 0.06159225 0.06159225]
 [0.29017094 0.23145957 0.33675214 0.09358974 0.09358974]
 [0.08333333 0.07307692 0.11666667 0.38888889 0.36111111]
 [0.08333333 0.07307692 0.11666667 0.36111111 0.38888889]]
PC at time step 1: [[0.33548365 0.25833464 0.27689401 0.06632072 0.06632072]
 [0.25872157 0.35814288 0.25565623 0.06386491 0.06386491]
 [0.23777254 0.21692026 0.34913409 0.09526426 0.09526426]
 [0.06666667 0.06153846 0.11904762 0.38888889 0.36111111]
 [0.06666667 0.06153846 0.11904762 0.36111111 0.38888889]]
P1 at time step 2: [[0.29180313 0.2859771  0.28021763 0.068394   0.068

### Part c)
Implement the SNF starting from the similarity matrices W(i) in Python with the convergence criterion ϵ as described in the supplement of the paper (Uni Tuebingen VPN necessary to access the full paper, supplement is freely available) and check whether the graph structure of the P(c)s changes for t > 2 for the above described data.

In [31]:
def joint_similarity_matrix(list_of_weight_matrices):
    num_row = list_of_weight_matrices[0].shape[0]
    num_col = list_of_weight_matrices[0].shape[1]
    num_matrices = len(list_of_weight_matrices)
    
    # Initialize P and S matrices
    P1 = normalized_weight_matrix(list_of_weight_matrices[0])
    P2 = normalized_weight_matrix(list_of_weight_matrices[1])
    S1 = similarity_matrix(list_of_weight_matrices[0])
    S2 = similarity_matrix(list_of_weight_matrices[1])

    # Initalize P and S at time  and P_c at time t
    P1_t = []
    P2_t = []
    PC_t = []

    # Fill lists for time = 0
    P1_t.append(P1)
    P2_t.append(P2)
    PC_t.append((P1_t[0] + P2_t[0])/2)

    # Calculate next time step

    # Initialize the stopping criterion epsilon from the suggestion of the paper
    epsilon = 10**(-6)

    i = 1
    stopping_criterion = 1
    
    while stopping_criterion > epsilon:
        P1_t.append(np.dot(S1, np.dot(P2_t[i-1], S1.T)))
        P2_t.append(np.dot(S2, np.dot(P1_t[i-1], S2.T)))
        PC_t.append((P1_t[i] + P2_t[i])/2)

        stopping_criterion = np.linalg.norm(PC_t[i] - PC_t[i - 1]) / np.linalg.norm(PC_t[i-1])
        
        print("PC_t at time " + str(i) + " :")
        print(PC_t[i])
        i += 1

joint_similarity_matrix([W1, W2])

PC_t at time 1 :
[[0.33548365 0.25833464 0.27689401 0.06632072 0.06632072]
 [0.25872157 0.35814288 0.25565623 0.06386491 0.06386491]
 [0.23777254 0.21692026 0.34913409 0.09526426 0.09526426]
 [0.06666667 0.06153846 0.11904762 0.38888889 0.36111111]
 [0.06666667 0.06153846 0.11904762 0.36111111 0.38888889]]
PC_t at time 2 :
[[0.28886169 0.27594225 0.28372832 0.07063565 0.07063565]
 [0.28107152 0.30424323 0.2831302  0.06911933 0.06911933]
 [0.26823259 0.26274103 0.29684518 0.08467782 0.08467782]
 [0.07686203 0.07148962 0.10091575 0.37654321 0.37345679]
 [0.07686203 0.07148962 0.10091575 0.37345679 0.37654321]]
PC_t at time 3 :
[[0.28362129 0.28125912 0.28518008 0.07337262 0.07337262]
 [0.28283126 0.29128104 0.28518095 0.07130271 0.07130271]
 [0.27563059 0.27410647 0.28648235 0.08033615 0.08033615]
 [0.07969068 0.07590871 0.09241816 0.37517147 0.37482853]
 [0.07969068 0.07590871 0.09241816 0.37482853 0.37517147]]
PC_t at time 4 :
[[0.28240852 0.28189187 0.28383356 0.0740915  0.0740915 ]
 

The algorithm does seem to change the weights of the graph for many more iterations, however the weights only change very slightly.